In [5]:
import cv2
import numpy as np
from random import shuffle
import os
from tqdm import tqdm

TRAIN_DIR = "/home/yeolpyeong/Dogs vs Cats/data/train"
TEST_DIR = "/home/yeolpyeong/Dogs vs Cats/data/test"
DATA_DIR = "/home/yeolpyeong/Dogs vs Cats/data/"
CODE_DIR = "/home/yeolpyeong/Dogs vs Cats/code/"
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = "dogsvscats-{}-{}.model".format(LR, "2conv-basic")

In [2]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save(DATA_DIR + 'train_data.npy', training_data)
    return training_data

In [6]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save(DATA_DIR + 'test_data.npy', testing_data)
    return testing_data
process_test_data()

100%|██████████| 12500/12500 [00:12<00:00, 983.66it/s] 


[[array([[150, 157, 158, ..., 109, 106, 106],
         [150, 156, 158, ..., 109, 107, 106],
         [149, 160, 160, ..., 110, 107, 106],
         ..., 
         [134, 134, 131, ...,  68,  65,  63],
         [251, 249, 252, ...,  89,  85,  83],
         [112, 111, 112, ...,  85,  81,  77]], dtype=uint8), '5605'],
 [array([[22, 22, 20, ...,  1,  1,  1],
         [20, 21, 18, ...,  2,  1,  1],
         [21, 20, 18, ..., 21, 18, 20],
         ..., 
         [11,  7, 11, ...,  7,  7,  6],
         [10,  8,  9, ...,  6,  3,  5],
         [13,  7, 10, ...,  5,  5,  3]], dtype=uint8), '11119'],
 [array([[196, 190, 171, ...,  90,  79,  71],
         [193, 182, 169, ...,  92, 101, 108],
         [191, 177, 163, ...,  94, 105, 111],
         ..., 
         [252, 245, 254, ..., 180, 169, 201],
         [186, 205, 241, ..., 243, 234, 174],
         [215, 221, 226, ..., 249, 249, 249]], dtype=uint8), '7995'],
 [array([[120, 128, 135, ..., 191, 190, 185],
         [126, 134, 141, ..., 195, 196, 191]

In [7]:
train_data = create_train_data()
# If you have already created the dataset:
#train_data = np.load(DATA_DIR + 'train_data.npy')

In [8]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [9]:
if os.path.exists(DATA_DIR + '{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

INFO:tensorflow:Restoring parameters from /home/yeolpyeong/dogsvscats-0.001-2conv-basic.model


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [256,1024] rhs shape= [32,1024]
	 [[Node: save_1/Assign_16 = Assign[T=DT_FLOAT, _class=["loc:@FullyConnected/W"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](FullyConnected/W, save_1/RestoreV2_16)]]

Caused by op 'save_1/Assign_16', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.

In [10]:
train = train_data[:-500]
test = train_data[-500:]

In [11]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [12]:
model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

---------------------------------
Run id: dogsvscats-0.001-2conv-basic.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 24500
Validation samples: 500
--


RuntimeError: Attempted to use a closed Session.

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('home/yeolpyeong/{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

joblib.dump(model, "model.pkl")
model.save(MODEL_NAME)

Training Step: 334  | total loss: 0.62920 | time: 92.143s
| Adam | epoch: 001 | loss: 0.62920 - acc: 0.6526 -- iter: 21376/24500


In [ ]:
import matplotlib.pyplot as plt

# if you need to create the data:
#test_data = process_test_data()
# if you already have some saved:
test_data = np.load(DATA_DIR + 'test_data.npy')

fig=plt.figure()

for num,data in enumerate(test_data[:12]):
    # cat: [1,0]
    # dog: [0,1]
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label='Dog'
    else: str_label='Cat'
        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
with open(DATA_DIR + 'submission_file.csv','w') as f:
    f.write('id,label\n')
            
with open(DATA_DIR + 'submission_file.csv','a') as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num,model_out[1]))